In [46]:
%reload_ext autoreload
%autoreload 2
from ce_builder import sensor_event_stream, watchbox, spatialEvent, \
    ce_and, ce_until, ce_follows, event_compile, complexEvent, Event
import time
import os

In [27]:
import json
data_file = open("complex_results.json", "r")
json_results = json.load(data_file)
data_file.close()

def get_data(frame_index):
    data = []
    
    for x in json_results:
        if x['time'] == frame_index:
            data.append(x)
    return data
    

In [5]:
import multiprocessing
def execute_visualization():
    os.system("python visualize.py")

In [47]:
# CE1

# First, initialize our complex event
ce1 = complexEvent()
# Set up our watchboxes
ce1.addWatchbox("bridgewatchbox1 = watchbox('camera3', positions=[213,274,772,772], id=0)")
ce1.addWatchbox("bridgewatchbox2 = watchbox('camera3', positions=[816,366,1200,725], id=1)")
ce1.addWatchbox("bridgewatchbox3 = watchbox('camera3', positions=[1294,290,1881,765], id=2)")

# Now we set up the atomic events

# First, four vehicles approach the bridge from one side
ev11a = Event("bridgewatchbox1.composition(at=0, model='rec_vehicle').size==4 and bridgewatchbox1.composition(at=1, model='rec_vehicle').size!=4")

# Next, we have two vehicles on either side of the bridge
ev11b = Event( "bridgewatchbox1.composition(at=0, model='rec_vehicle').size==2 and bridgewatchbox3.composition(at=0, model='rec_vehicle').size==2" )

# Then we have two vehicles exit watchbox 3
ev11c1 = Event( "bridgewatchbox3.composition(at=1, model='rec_vehicle').size==2 and bridgewatchbox3.composition(at=0, model='rec_vehicle').size==0" )
#  Possible transitions 2-1-0, or 2-0 - two possible sequences have to be considered and either can be true

# And then two vehicles show up in watchbox 1
ev11c2 = Event( "bridgewatchbox1.composition(at=0, model='rec_vehicle').size==4 and bridgewatchbox1.composition(at=1, model='rec_vehicle').size!=4" )

# And finally we add these events together
ce1.addEventSequence([ ev11a, ev11b, ev11c1, ev11c2 ])

# Run a visualization
# multiprocessing.Process(target=execute_visualization).start()
# time.sleep(1.5) # Visualization takes some time to set up

# Now we can iterate through some frames
for frame_index in range(0, 60*30):
    
    # Parse our result data, and display it at the correct frame index
    incoming_data = get_data(frame_index)
    if incoming_data:
        
        #### RUN OUR EVALUATION ON THE EVENT ANYTIME WE GET NEW DATA
        
        ce1.update(incoming_data)
        
        event_occurred, event_name = ce1.evaluate()
        if event_occurred:
            print("Event %s occurred at frame %d" %(event_name, frame_index))
        
    
    # time.sleep(1/30)





Event ev11a occurred at frame 652
Event ev11b occurred at frame 799
Event ev11c1 occurred at frame 897
Event ev11c2 occurred at frame 983
